In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score, f1_score

In [5]:
df = pd.read_csv('anime.csv')

In [63]:
df

anime_id                                               name  \
0         32281                                     Kimi no Na wa.   
1          5114                   Fullmetal Alchemist: Brotherhood   
2         28977                                           Gintama°   
3          9253                                        Steins;Gate   
4          9969                                      Gintama&#039;   
...         ...                                                ...   
12289      9316       Toushindai My Lover: Minami tai Mecha-Minami   
12290      5543                                        Under World   
12291      5621                     Violence Gekiga David no Hoshi   
12292      6133  Violence Gekiga Shin David no Hoshi: Inma Dens...   
12293     26081                   Yasuji no Pornorama: Yacchimae!!   

                                                   genre   type episodes  \
0                   Drama, Romance, School, Supernatural  Movie        1   
1      Action, Adventure, Drama, Fantasy, Magic, Mili...     TV       64   
2      Action, Comedy, Historical, Parody, Samurai, S...     TV       51   
3                                       Sci-Fi, Thriller     TV       24   
4      Action, Comedy, Historical, Parody, Samurai, S...     TV       51   
...                                                  ...    ...      ...   
12289                                             Hentai    OVA        1   
12290                                             Hentai    OVA        1   
12291                                             Hentai    OVA        4   
12292                                             Hentai    OVA        1   
12293                                             Hentai  Movie        1   

       rating  members  
0        9.37   200630  
1        9.26   793665  
2        9.25   114262  
3        9.17   673572  
4        9.16   151266  
...       ...      ...  
12289    4.15      211  
12290    4.28      183  
12291    4.88      219  
12292    4.98      175  
12293    5.46      142  

[12294 rows x 7 columns]

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [7]:
# Check for missing values
print(df.isnull().sum())

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64


In [10]:
# Handle missing values
# Fill missing values in 'genre' with 'Unknown'
df['genre'] = df['genre'].fillna('Unknown')
# Fill missing values in 'type' with 'Unknown'
df['type'] = df['type'].fillna('Unknown')
# Fill missing values in 'rating' with the mean rating
df['rating'] = df['rating'].fillna(df['rating'].mean())

In [11]:
# Check for missing values
print(df.isnull().sum())

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64


In [13]:
# Display summary statistics
print(df.describe())

# Display data types of each column
print(df.dtypes)

           anime_id        rating       members
count  12294.000000  12294.000000  1.229400e+04
mean   14058.221653      6.473902  1.807134e+04
std    11455.294701      1.017096  5.482068e+04
min        1.000000      1.670000  5.000000e+00
25%     3484.250000      5.900000  2.250000e+02
50%    10260.500000      6.550000  1.550000e+03
75%    24794.500000      7.170000  9.437000e+03
max    34527.000000     10.000000  1.013917e+06
anime_id      int64
name         object
genre        object
type         object
episodes     object
rating      float64
members       int64
dtype: object


In [15]:
# Convert 'genre' column into a list of genres
df['genre'] = df['genre'].apply(lambda x: x.split(', ') if x != 'Unknown' else [])


In [18]:
# Use MultiLabelBinarizer to one-hot encode the genres
mlb = MultiLabelBinarizer()
genre_encoded = mlb.fit_transform(df['genre'])
genre_df = pd.DataFrame(genre_encoded, columns=mlb.classes_)

In [20]:
# Normalize the 'rating' column
scaler = StandardScaler()
df['rating_normalized'] = scaler.fit_transform(df[['rating']])

In [21]:
# Combine the genre and rating features into a single DataFrame
features_df = pd.concat([genre_df, df[['rating_normalized']]], axis=1)

In [22]:
# Display the first few rows of the features DataFrame
print(features_df.head())

   Action  Adventure  Cars  Comedy  Dementia  Demons  Drama  Ecchi  Fantasy  \
0       0          0     0       0         0       0      1      0        0   
1       1          1     0       0         0       0      1      0        1   
2       1          0     0       1         0       0      0      0        0   
3       0          0     0       0         0       0      0      0        0   
4       1          0     0       1         0       0      0      0        0   

   Game  ...  Slice of Life  Space  Sports  Super Power  Supernatural  \
0     0  ...              0      0       0            0             1   
1     0  ...              0      0       0            0             0   
2     0  ...              0      0       0            0             0   
3     0  ...              0      0       0            0             0   
4     0  ...              0      0       0            0             0   

   Thriller  Vampire  Yaoi  Yuri  rating_normalized  
0         0        0     0     0

In [23]:
from sklearn.metrics.pairwise import cosine_similarity

# Function to recommend anime based on cosine similarity
def recommend_anime(target_anime_name, features_df, df, top_n=10):
    # Find the index of the target anime
    target_index = df[df['name'] == target_anime_name].index[0]
    
    # Compute cosine similarity between the target anime and all other anime
    similarity_scores = cosine_similarity(features_df.iloc[target_index:target_index+1], features_df).flatten()
    
    # Get the indices of the top_n most similar anime (excluding the target anime itself)
    similar_indices = similarity_scores.argsort()[::-1][1:top_n+1]
    
    # Get the names and similarity scores of the recommended anime
    recommended_anime = df.iloc[similar_indices][['name', 'rating']]
    recommended_anime['similarity_score'] = similarity_scores[similar_indices]
    
    return recommended_anime

# Example usage: recommend anime similar to 'Naruto'
recommended_anime = recommend_anime('Naruto', features_df, df)
print(recommended_anime)

                                                   name  rating  \
615                                  Naruto: Shippuuden    7.94   
1103  Boruto: Naruto the Movie - Naruto ga Hokage ni...    7.68   
486                            Boruto: Naruto the Movie    8.03   
1343                                        Naruto x UT    7.58   
1472        Naruto: Shippuuden Movie 4 - The Lost Tower    7.53   
1573  Naruto: Shippuuden Movie 3 - Hi no Ishi wo Tsu...    7.50   
2458               Naruto Shippuuden: Sunny Side Battle    7.26   
2997  Naruto Soyokazeden Movie: Naruto to Mashin to ...    7.11   
175                              Katekyo Hitman Reborn!    8.37   
206                                       Dragon Ball Z    8.32   

      similarity_score  
615           0.999142  
1103          0.999052  
486           0.997628  
1343          0.996921  
1472          0.995354  
1573          0.994245  
2458          0.980388  
2997          0.966788  
175           0.909505  
206         

In [28]:
# Split the dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Function to evaluate the recommendation system
def evaluate_recommendation_system(recommendation_function, test_df, features_df, df, top_n=10):
    y_true = []
    y_pred = []

    for target_anime_name in test_df['name']:
        try:
            # Get the index of the target anime
            target_index = df[df['name'] == target_anime_name].index[0]

            # Get the actual similar anime (ground truth)
            similarity_scores = cosine_similarity(features_df.iloc[target_index:target_index+1], features_df).flatten()
            actual_similar_indices = similarity_scores.argsort()[::-1][1:top_n+1]
            actual_similar_anime = df.iloc[actual_similar_indices]['name'].tolist()
            
            # Get the recommended anime
            recommended_anime = recommendation_function(target_anime_name, features_df, df, top_n)['name'].tolist()

            # Append the actual and predicted values
            y_true.extend(actual_similar_anime)
            y_pred.extend(recommended_anime)
        except IndexError:
            print(f"Anime '{target_anime_name}' not found in the dataset.")
            continue

    # Calculate precision, recall, and F1-score
    precision = precision_score(y_true, y_pred, average='micro', zero_division=0)
    recall = recall_score(y_true, y_pred, average='micro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='micro', zero_division=0)
    
    return precision, recall, f1

# Evaluate the recommendation system
precision, recall, f1 = evaluate_recommendation_system(recommend_anime, test_df, features_df, df)
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')

Precision: 1.0
Recall: 1.0
F1-score: 1.0


In [ ]:
"""What is collaborative filtering, and how does it work?
    Collaborative filtering is a popular technique used in recommendation systems to predict the preferences of a user 
    by collecting preferences from many users. The fundamental assumption behind collaborative filtering is that if 
    users agree on certain items, they will likely agree on other items as well. It can be divided into two main types:
    user-based collaborative filtering and item-based collaborative filtering.
    
     User-Based Collaborative Filtering
User-based collaborative filtering makes recommendations based on the similarity between users. The idea is to find users 
similar to the target user and recommend items that those similar users have liked.

    Item-Based Collaborative Filtering
Item-based collaborative filtering makes recommendations based on the similarity between items. The idea is to find items 
similar to the ones the target user has liked and recommend those items.

In [ ]:
""" Can you explain the difference between user-based and item-based collaborative filtering?
User-based collaborative filtering focuses on the relationships between users. The core idea is that if two 
users have similar preferences, then the items liked by one user can be recommended to the other user.
Identify Similar Users: Calculate the similarity between users based on their past behavior (e.g., ratings, likes).
Select Neighbors: Choose a set of similar users (neighbors) for the target user.
Generate Recommendations: Recommend items that the similar users have liked but the target user has not yet interacted with.
"""

"""Item-based collaborative filtering focuses on the relationships between items. The core idea is that if a user likes 
certain items, they are likely to like similar items.

Key Steps:
Identify Similar Items: Calculate the similarity between items based on user ratings or interactions.
Generate Recommendations: Recommend items that are similar to those the user has already liked."""